# Regime Probability Field Engine — Demo

**Architecture**: `OHLCV (1m) → Features → KPCOFGS Taxonomy → Log-space Probability Field → 28 Species → 8 Regime Bins → Validity-Gated Indicators → AR(1) Projection → Composite Score`

## What this system does

| Stage | Method | Key Constraint |
|-------|--------|----------------|
| Regime Inference | Hierarchical energy model (KPCOFGS) | Pure statistical features — no indicator leakage |
| Probability propagation | Log-space via `logsumexp` | Numerically stable, no underflow |
| Species assignment | 28 fixed leaves, mutually exclusive | Fully probabilistic, no hard thresholds |
| Indicator gating | Soft weight from species P(n) | Validity set per indicator |
| Signal projection | Regime-conditioned AR(1) mixture | 8 collapsed regime bins |
| Composite score | Weighted average × 3 confidence metrics | C_field × C_consensus × C_liquidity |

In [ ]:
import sys
import pathlib

# Add project root to path
project_root = pathlib.Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
import warnings
warnings.filterwarnings('ignore')

# Regime engine imports
from regime_engine import (
    RegimeEngine, UniverseScanner, simulate_ohlcv, load_config,
    SPECIES_LIST, REGIME_BINS
)

print('Regime Probability Field Engine loaded successfully.')
print(f'Species count: {len(SPECIES_LIST)} (fixed 28-leaf taxonomy)')
print(f'Regime bins:   {REGIME_BINS}')

## 1. Simulate 1-minute OHLCV Data

The simulator injects deliberate regime transitions:
- **Bars 0–25%**: Trend (random direction, persistent drift)
- **Bars 25–50%**: Range (AR(1) mean-reverting)
- **Bars 50–75%**: Trend again
- **Bars 75–100%**: Breakout (squeeze then explosive)

In [ ]:
ohlcv = simulate_ohlcv(n_bars=1200, seed=7, regime_sequence=True)
print(f'Shape: {ohlcv.shape}')
print(ohlcv.head(3).to_string())
print(f'\nDate range: {ohlcv.index[0]} → {ohlcv.index[-1]}')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
fig.suptitle('Simulated 1-Minute OHLCV', fontsize=14)

axes[0].plot(ohlcv.index, ohlcv['close'], color='steelblue', lw=0.8)
axes[0].set_ylabel('Price')
axes[0].set_title('Close Price — Regime Sequence: Trend → Range → Trend → Breakout')

# Shade regime regions
n = len(ohlcv)
colors = ['#b8d4f5', '#f5dbb8', '#b8d4f5', '#f5b8c4']
labels = ['Trend', 'Range', 'Trend', 'Breakout']
for i, (col, lbl) in enumerate(zip(colors, labels)):
    start = ohlcv.index[n * i // 4]
    end   = ohlcv.index[min(n * (i + 1) // 4 - 1, n - 1)]
    axes[0].axvspan(start, end, alpha=0.3, color=col, label=lbl)
axes[0].legend(loc='upper left', fontsize=8)

axes[1].bar(ohlcv.index, ohlcv['volume'], width=0.0007, color='gray', alpha=0.6)
axes[1].set_ylabel('Volume')

plt.tight_layout()
plt.show()

## 2. Run the Full Engine

In [ ]:
cfg    = load_config()
engine = RegimeEngine(cfg)
result = engine.run(ohlcv)

print('Output keys:', list(result.keys()))
print(f"Features shape:     {result['features'].shape}")
print(f"Logits shape:       {result['logits'].shape}")
print(f"Node log-probs cols:{list(result['node_log_probs'].columns[:8])} ...")
print(f"Species LP shape:   {result['species_lp'].shape}")
print(f"Regime probs shape: {result['regime_probs'].shape}")
print(f"Mix shape:          {result['mix'].shape}")

## 3. Feature Engine Output

In [ ]:
features = result['features']
warmup = 100  # skip NaN-heavy early bars

fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Robust-Normalized Features (z-score)', fontsize=13)

feat_groups = [
    (['drift_tscore_15', 'drift_tscore_30', 'drift_tscore_60'],
     'Drift T-Scores (trend strength)'),
    (['rv_30', 'rv_120', 'rv_delta'],
     'Realized Volatility & ΔRV'),
    (['er_30', 'er_60', 'er_120'],
     'Efficiency Ratio (directional purity)'),
    (['autocorr', 'flip_rate', 'impulse_revert', 'entropy'],
     'Microstructure Features'),
]

idx = features.index[warmup:]
for ax, (cols, title) in zip(axes, feat_groups):
    for col in cols:
        if col in features.columns:
            ax.plot(idx, features[col].iloc[warmup:], lw=0.7, label=col, alpha=0.85)
    ax.axhline(0, color='black', lw=0.5, ls='--')
    ax.set_title(title, fontsize=10)
    ax.legend(fontsize=8, loc='upper right')

plt.tight_layout()
plt.show()

## 4. KPCOFGS Taxonomy — Kingdom & Phylum Probabilities

In [ ]:
node_lp = result['node_log_probs']

# Convert log-probs to linear probs
def lp_to_p(cols):
    lp_arr = node_lp[cols].clip(upper=0).values
    p = np.exp(lp_arr)
    return p / p.sum(axis=1, keepdims=True).clip(min=1e-15)

k_probs = lp_to_p(['DIR', 'NDR', 'TRN'])
v_probs = lp_to_p(['LV', 'NV', 'HV'])

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
fig.suptitle('Taxonomy Probability Field — Kingdom & Phylum', fontsize=13)

idx = node_lp.index[warmup:]

# Kingdom stacked area
kd = k_probs[warmup:]
axes[0].stackplot(idx, kd[:, 0], kd[:, 1], kd[:, 2],
                  labels=['DIR (Directional)', 'NDR (Non-directional)', 'TRN (Transitional)'],
                  colors=['#4e8caf', '#e8a45a', '#7cbb6e'], alpha=0.85)
axes[0].set_ylabel('Probability')
axes[0].set_title('Kingdom Probability (DIR / NDR / TRN)')
axes[0].legend(loc='upper left', fontsize=9)
axes[0].set_ylim(0, 1)

# Phylum stacked area
vd = v_probs[warmup:]
axes[1].stackplot(idx, vd[:, 0], vd[:, 1], vd[:, 2],
                  labels=['LV (Low Vol)', 'NV (Normal Vol)', 'HV (High Vol)'],
                  colors=['#a8d8ea', '#a8e6cf', '#f4a261'], alpha=0.85)
axes[1].set_ylabel('Probability')
axes[1].set_title('Phylum Probability (LV / NV / HV)')
axes[1].legend(loc='upper left', fontsize=9)
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 5. 8-Bin Collapsed Regime Probabilities

In [ ]:
regime_probs = result['regime_probs']

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
fig.suptitle('8-Bin Collapsed Regime Probability Field', fontsize=13)

idx = regime_probs.index[warmup:]
rp  = regime_probs.iloc[warmup:]

# Directional regimes
dir_cols = ['TREND_UP', 'TREND_DN', 'BREAKOUT_UP', 'BREAKOUT_DN', 'EXHAUST_REV']
dir_colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12']
for col, color in zip(dir_cols, dir_colors):
    axes[0].plot(idx, rp[col], label=col, color=color, lw=1.0, alpha=0.9)
axes[0].set_ylabel('Probability')
axes[0].set_title('Directional + Transitional Regimes')
axes[0].legend(fontsize=8, loc='upper right')

# Vol + Range regimes
other_cols = ['RANGE', 'LOWVOL', 'HIGHVOL']
other_colors = ['#95a5a6', '#2980b9', '#c0392b']
for col, color in zip(other_cols, other_colors):
    axes[1].plot(idx, rp[col], label=col, color=color, lw=1.0, alpha=0.9)
axes[1].set_ylabel('Probability')
axes[1].set_title('Range + Volatility Regimes')
axes[1].legend(fontsize=8, loc='upper right')

plt.tight_layout()
plt.show()

## 6. Species Probabilities — Top 10 Active Species

In [ ]:
species_lp = result['species_lp']
species_p  = np.exp(species_lp.clip(upper=0))

# Find top-10 species by mean probability in the latter half
mean_p = species_p.iloc[species_p.shape[0]//2:].mean()
top10  = mean_p.nlargest(10).index.tolist()

# Descriptions
sp_map = {s.id: s.description for s in SPECIES_LIST}

fig, ax = plt.subplots(figsize=(14, 5))
ax.set_title('Top-10 Species Probabilities (latter half of series)', fontsize=12)

idx = species_p.index[warmup:]
for sid in top10:
    ax.plot(idx, species_p[sid].iloc[warmup:],
            label=f"{sid}: {sp_map[sid][:45]}...", lw=1.0, alpha=0.85)

ax.set_ylabel('P(species)')
ax.legend(fontsize=7, loc='upper right', ncol=1)
plt.tight_layout()
plt.show()

# Most probable species at last bar
last_p = species_p.iloc[-1].sort_values(ascending=False)
print('\nTop 5 species at LAST BAR:')
for sid, p in last_p.head(5).items():
    sp = next(s for s in SPECIES_LIST if s.id == sid)
    print(f'  {sid}  P={p:.4f}  [{sp.kingdom}.{sp.phylum}.{sp.class_}]  {sp.description}')

## 7. Indicator Signals + Validity Weights

Each indicator has a soft validity weight `w_j = Σ_{n ∈ V_j} P(n)`.
Only active when the relevant regime is probable.

In [ ]:
signals = result['signals']
weights = result['weights']

print('Active indicators:', list(signals.columns))
print('\nMean validity weights (last 200 bars):')
print(weights.iloc[-200:].mean().sort_values(ascending=False).to_string())

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
fig.suptitle('Indicator Signals & Validity Weights', fontsize=13)

idx = signals.index[warmup:]

# Signals
for col in signals.columns:
    axes[0].plot(idx, signals[col].iloc[warmup:], lw=0.7, label=col, alpha=0.75)
axes[0].axhline(0, color='black', lw=0.5, ls='--')
axes[0].set_ylabel('Normalized Signal')
axes[0].set_title('Indicator Signals')
axes[0].legend(fontsize=7, loc='upper right', ncol=2)

# Weights
for col in weights.columns:
    axes[1].plot(idx, weights[col].iloc[warmup:], lw=0.7, label=col, alpha=0.75)
axes[1].set_ylabel('Validity Weight w_j')
axes[1].set_title('Soft Validity Weights (higher = indicator more relevant)')
axes[1].legend(fontsize=7, loc='upper right', ncol=2)

plt.tight_layout()
plt.show()

## 8. AR(1) Mixture Projections

Each indicator signal is projected one step forward under the regime-conditioned AR(1) mixture.

In [ ]:
proj_exp = result['projections']['expected']
proj_var = result['projections']['variance']
proj_std = np.sqrt(proj_var.clip(lower=0))

# Show MACD projection as example
indicators_to_show = [c for c in signals.columns if c in proj_exp.columns][:3]

fig, axes = plt.subplots(len(indicators_to_show), 1,
                         figsize=(14, 4 * len(indicators_to_show)), sharex=True)
if len(indicators_to_show) == 1:
    axes = [axes]

fig.suptitle('Regime-Conditioned AR(1) Projections (±1σ)', fontsize=13)

idx = signals.index[warmup:]

for ax, col in zip(axes, indicators_to_show):
    sig  = signals[col].iloc[warmup:]
    exp  = proj_exp[col].iloc[warmup:]
    std  = proj_std[col].iloc[warmup:]

    ax.plot(idx, sig,  color='steelblue',  lw=0.8, label='Current signal', alpha=0.9)
    ax.plot(idx, exp,  color='darkorange', lw=1.0, label='Projected E[x_{t+1}]', ls='--')
    ax.fill_between(idx, (exp - std).values, (exp + std).values,
                    color='darkorange', alpha=0.15, label='±1σ mixture')
    ax.axhline(0, color='black', lw=0.5, ls=':')
    ax.set_title(f'{col} — current vs projected')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 9. Composite Score & Confidence Decomposition

In [ ]:
mix = result['mix']

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Composite Score & Confidence Decomposition', fontsize=13)

idx = mix.index[warmup:]
mx  = mix.iloc[warmup:]

# Composite signal
axes[0].bar(idx, mx['composite_signal'],
            color=np.where(mx['composite_signal'] > 0, '#2ecc71', '#e74c3c'),
            width=0.0006, alpha=0.8)
axes[0].axhline(0, color='black', lw=0.7)
axes[0].set_title('Composite Signal S_t (validity-weighted indicator average)')
axes[0].set_ylabel('Signal')

# Confidence metrics
axes[1].plot(idx, mx['c_field'],     lw=1.0, label='C_field (taxonomy entropy)', color='royalblue')
axes[1].plot(idx, mx['c_consensus'], lw=1.0, label='C_consensus (signal agreement)', color='seagreen')
axes[1].plot(idx, mx['c_liquidity'], lw=1.0, label='C_liquidity (vol/gap condition)', color='darkorange')
axes[1].set_title('Confidence Metrics (all ∈ [0,1])')
axes[1].set_ylabel('Confidence')
axes[1].legend(fontsize=9)

# Final score
axes[2].bar(idx, mx['score'],
            color=np.where(mx['score'] > 0, '#1a9850', '#d73027'),
            width=0.0006, alpha=0.85)
axes[2].axhline(0, color='black', lw=0.7)
axes[2].set_title('Final Score = S_t × C_field × C_consensus × C_liquidity')
axes[2].set_ylabel('Score')

plt.tight_layout()
plt.show()

print('Last 5 bars:')
print(mix.tail(5).to_string())

## 10. Universe Scanner Demo

In [ ]:
# Simulate a universe of 10 synthetic tickers
tickers = ['AAPL','MSFT','NVDA','TSLA','SPY','QQQ','META','AMZN','GOOGL','AMD']
universe = {t: simulate_ohlcv(n_bars=800, seed=hash(t) % 100) for t in tickers}

scanner = UniverseScanner()
ranked  = scanner.scan(universe)

print('=== UNIVERSE SCAN RESULTS ===')
display_cols = [
    'ticker', 'score', 'composite_signal',
    'c_field', 'c_consensus', 'c_liquidity',
    'top_species', 'top_species_desc',
    'p_TREND_UP', 'p_TREND_DN', 'p_RANGE', 'p_BREAKOUT_UP'
]
available = [c for c in display_cols if c in ranked.columns]
print(ranked[available].to_string(index=False))

In [ ]:
# Filter by specific regime
print('\n=== TICKERS IN HIGH RANGE PROBABILITY ===')
range_scan = scanner.scan_top_regime(universe, regime='RANGE', min_prob=0.20)
if len(range_scan):
    print(range_scan[['ticker','p_RANGE','score','top_species','top_species_desc']].to_string(index=False))
else:
    print('No tickers above threshold')

## 11. Indicator Contribution Attribution

Diagnostic: which indicators are driving the composite signal?

In [ ]:
contrib = engine.mixer.indicator_contributions(signals, weights)

fig, ax = plt.subplots(figsize=(14, 4))
idx = contrib.index[warmup:]

# Stacked bar chart of contributions
bottom_pos = pd.Series(0.0, index=idx)
bottom_neg = pd.Series(0.0, index=idx)
colors_palette = plt.cm.Set2(np.linspace(0, 1, len(contrib.columns)))

for col, color in zip(contrib.columns, colors_palette):
    vals = contrib[col].iloc[warmup:]
    pos  = vals.clip(lower=0)
    neg  = vals.clip(upper=0)
    ax.bar(idx, pos, bottom=bottom_pos, color=color, alpha=0.8, label=col, width=0.0006)
    ax.bar(idx, neg, bottom=bottom_neg, color=color, alpha=0.8, width=0.0006)
    bottom_pos = bottom_pos + pos
    bottom_neg = bottom_neg + neg

ax.axhline(0, color='black', lw=0.7)
ax.set_title('Indicator Contribution to Composite Signal (by indicator)')
ax.set_ylabel('Contribution')
ax.legend(fontsize=7, loc='upper right', ncol=2)
plt.tight_layout()
plt.show()

## 12. AI Tuning Interface

The system is designed for AI parameter optimization.

### Tunable parameters:
- **`config.yaml taxonomy.energy.*`** — Alpha coefficients control which features drive each taxonomy node
- **`config.yaml taxonomy.smoothing.*`** — EWM alpha controls how sticky each level is (higher = more responsive)
- **`config.yaml projection.regimes.*`** — AR(1) parameters (μ, φ, β, σ) per regime

### Tuning objective:
Maximize out-of-sample prediction accuracy of `proj_exp` vs `signal[t+1]`

In [ ]:
# Example: inspect current AR(1) projection parameters
proj_engine = engine.projection
params = proj_engine.get_params_dict()
print('Current AR(1) regime parameters:')
df_params = pd.DataFrame(params).T
print(df_params.to_string())

print()
print('To update (AI tuning interface):')
print('  proj_engine.set_params_dict({')
print('      "TREND_UP": {"mu": 0.4, "phi": 0.92, "beta": 0.12, "sigma": 0.14},')
print('      ...')
print('  })')

In [ ]:
# Projection accuracy diagnostic — 1-step ahead
# For each indicator, compare projected[t] vs actual signal[t+1]

proj_exp  = result['projections']['expected']
actual_next = signals.shift(-1)  # actual t+1 value

warmup_trim = 200
errors = (proj_exp.iloc[warmup_trim:] - actual_next.iloc[warmup_trim:])
rmse   = np.sqrt((errors**2).mean())
print('1-step-ahead RMSE per indicator (lower = better projection):')
print(rmse.sort_values().to_string())
print(f'\nMean RMSE across all indicators: {rmse.mean():.4f}')
print('\n(Tuning projection AR(1) parameters can reduce this RMSE)')

## Summary

| Component | Status | Notes |
|-----------|--------|-------|
| Feature Engine | ✓ Vectorized | 20 features, robust z-score normalized |
| Taxonomy (KPCOFGS) | ✓ Probabilistic | Log-space sticky logits via EWM |
| Species Field | ✓ 28 leaves | Log-space propagation, logsumexp normalized |
| Regime Bins | ✓ 8 bins | Drift-sign split for directional regimes |
| Indicator Library | ✓ 11 indicators | Types A/B/C/D with validity sets |
| Projection Engine | ✓ AR(1) mixture | Per-regime μ/φ/β/σ, AI-tunable |
| Composite Score | ✓ Full | C_field × C_consensus × C_liquidity |
| Scanner | ✓ Universe | Sort by any metric, filter by regime |

### Design guarantees:
- **No indicator leakage** into regime inference — features are purely statistical
- **No hard thresholds** — all transitions probabilistic
- **Numerically stable** — log-space propagation throughout
- **Modular** — each stage independently testable
- **AI-tunable** — all alpha coefficients exposed via config.yaml